# Homework 2 Review

I am among the sad souls that had done the homework, but lost everything during the cluster login node crash. But since I'm the instructor, I am allowed to not re-do the homework. Instead I decided to put this notebook together for everyone. It has some similarities to the homework (it extracts sub-images/image chips) using bounding box data. Instead of reading and writing dataframes to files I added a section of how to run the image chips through a Keras model on the cluster.

# Rework of data603 library .ipynb files

## HDFS.ipynb

I updated this notebook with another way to use HDFS. While doing this exercise I was unable to get `pyarrow` configured correctly on the worker nodes, _however_ I was able to use the HDFS HTTP interface to connect. See the notebook for details and a link to the API. The other methods are still there, but if you want a udf to connect to HDFS, you'll need to borrow code from the HDFS.ipynb. Here I use this new interface to list files in HDFS. 

In [1]:
import import_ipynb
from data603 import HDFS

httpdfs = HDFS.get_httpdfs()
httpdfs.list('/data/keras_models')

importing Jupyter notebook from /scratch/data603/klucar/data603/HDFS.ipynb


['densenet',
 'efficientnet',
 'inception_resnet_v2',
 'inception_v3',
 'mobilenet',
 'mobilenet_v2',
 'mobilenet_v3',
 'nasnet',
 'resnet',
 'vgg16',
 'vgg19',
 'xception']

# Keras Weight Files

I took the opportunity to download all the built-in Keras models weight files. The details of the file naming convention are beyond this demonstration, but reach out and I can help you, or read the Keras code because that's what I did! The directory names are the names of the various models built in to Keras. Each directory contains several .h5 weight files for each model. The models have differrent parameters such as input image size, that vary with each model. For this demonstration I'll use the `mobilenet` model because it is optimized to run on mobile hardware, so it should run faster than other models.

In [2]:
httpdfs.list('/data/keras_models/mobilenet')

['mobilenet_1_0_128_tf.h5',
 'mobilenet_1_0_128_tf_no_top.h5',
 'mobilenet_1_0_160_tf.h5',
 'mobilenet_1_0_160_tf_no_top.h5',
 'mobilenet_1_0_192_tf.h5',
 'mobilenet_1_0_192_tf_no_top.h5',
 'mobilenet_1_0_224_tf.h5',
 'mobilenet_1_0_224_tf_no_top.h5',
 'mobilenet_2_5_128_tf.h5',
 'mobilenet_2_5_128_tf_no_top.h5',
 'mobilenet_2_5_160_tf.h5',
 'mobilenet_2_5_160_tf_no_top.h5',
 'mobilenet_2_5_192_tf.h5',
 'mobilenet_2_5_192_tf_no_top.h5',
 'mobilenet_2_5_224_tf.h5',
 'mobilenet_2_5_224_tf_no_top.h5',
 'mobilenet_5_0_128_tf.h5',
 'mobilenet_5_0_128_tf_no_top.h5',
 'mobilenet_5_0_160_tf.h5',
 'mobilenet_5_0_160_tf_no_top.h5',
 'mobilenet_5_0_192_tf.h5',
 'mobilenet_5_0_192_tf_no_top.h5',
 'mobilenet_5_0_224_tf.h5',
 'mobilenet_5_0_224_tf_no_top.h5',
 'mobilenet_7_5_128_tf.h5',
 'mobilenet_7_5_128_tf_no_top.h5',
 'mobilenet_7_5_160_tf.h5',
 'mobilenet_7_5_160_tf_no_top.h5',
 'mobilenet_7_5_192_tf.h5',
 'mobilenet_7_5_192_tf_no_top.h5',
 'mobilenet_7_5_224_tf.h5',
 'mobilenet_7_5_224_tf_no_t

The file I need is the `mobilenet_1_0_224_tf.h5` This is the weight file for 224x224 pixel input images and alpha of 1.0, meaning use full-size filters in the hidden layers. The _no_top_ files don't have the final output layer and are meant to be used to re-train the models with your own labels.

# Download Weight File

In [3]:
# Create a local directory
import os
keras_data = './keras_data'
if(not os.path.exists(keras_data)):
    os.mkdir(keras_data)

# download file from hdfs
mobilenet_weight_file = 'mobilenet_1_0_224_tf.h5'
local_weight_file = f"{keras_data}/{mobilenet_weight_file}"
if(not os.path.exists(local_weight_file)):
    httpdfs.download(f"/data/keras_models/mobilenet/{mobilenet_weight_file}", local_weight_file)

# check local file exists
os.listdir(keras_data)

['mobilenet_1_0_224_tf.h5',
 'imagenet_class_index.json',
 'bbox_labels_600_hierarchy.json',
 '.ipynb_checkpoints',
 'Ostrich_f20950a7e7db32d4_344161.jpeg',
 'Magpie_387d01e608451323_784664.jpeg',
 'Goose_f7fe3c8c98e5770d_190299.jpeg']

## SparkLauncher.ipynb

Instead of constantly handing out new versions of this notebook (too late, I know) I decided to break the function into two functions. The first function returns a configuration object and the second takes this configuration as input to create a spark.sql session. This allows you the opportunity to add your own configuration items before launching Spark, but it is backward compatible with the older versions so your old notebooks should still work without any code updates. I use this to distribute the .h5 weight file to each spark node.

In [4]:
import import_ipynb
from data603 import SparkLauncher

# get a configuration object
conf = SparkLauncher.get_spark_conf()

# add a file to the configuration that will get copied to all the nodes on the cluster
conf.set('spark.yarn.dist.files', './keras_data/mobilenet_1_0_224_tf.h5')

# launch the cluster using the configuration
spark = SparkLauncher.get_spark_session(pack_venv = False, conf = conf)

importing Jupyter notebook from /scratch/data603/klucar/data603/SparkLauncher.ipynb
Creating Spark Configuration
Creating Spark Configuration
Setting Environment Variables
Creating Spark Session: klucar_data603_spark_session


# Extracting Image Chips

Now that Spark is running and it has our model weight file, we need to extract sub-images from the google open image dataset. This is the part that is very similar to homework 2.

In [5]:
import os
import pyspark.sql.functions as F
from pyspark.sql.types import *

## Find Some Interesting Chips

Most of the Keras Models use ResNet image labels. ResNet has 1000 labels. Here are some labels of birds that are common between the two datasets:

|Google OID | ResNet |
|-----------|--------|
| Magpie | magpie |
| Ostrich | ostrich |
| Goose | goose |

I searched the OID labels by examining the OID label hierarchy [here.](https://storage.googleapis.com/openimages/2018_04/bbox_labels_600_hierarchy_visualizer/circle.html) It's also available for download in JSON format [here.](https://storage.googleapis.com/openimages/2018_04/bbox_labels_600_hierarchy.json)

I downloaded the Resnet labels from [Kaggle](https://www.kaggle.com/alexisbcook/resnet50).

### Load Class Descriptions Into Spark Dataframe

Example File Format, no header

```
...
/m/0pc9,Alphorn
/m/0pckp,Robin
/m/0pcm_,Larch
/m/0pcq81q,Soccer player
/m/0pcr,Alpaca
/m/0pcvyk2,Nem
/m/0pd7,Army
/m/0pdnd2t,Bengal clockvine
/m/0pdnpc9,Bushwacker
/m/0pdnsdx,Enduro
/m/0pdnymj,Gekkonidae
...
```

In [6]:
from pyspark.sql.types import *

labels = spark.read.csv('/data/google_open_image/metadata/class-descriptions-boxable.csv', 
                        header = False,
                        schema = StructType([StructField("LabelName", StringType()), 
                                             StructField("LabelText", StringType())]) )

# Filter labels to just the three we're interested in _Magpie_ _Ostrich_ and _Goose_

In [7]:
labels = labels.filter("LabelText = 'Magpie' OR LabelText = 'Ostrich' OR LabelText = 'Goose'")

# since it's just 3 labels, bring it back as a pandas dataframe
pd_labels = labels.toPandas()
pd_labels

,LabelName,LabelText
0,/m/012074,Magpie
1,/m/05n4y,Ostrich
2,/m/0dbvp,Goose


# Download Image IDs with their labels

File Format Example
```
ImageID,Source,LabelName,Confidence
000026e7ee790996,verification,/m/04hgtk,0
000026e7ee790996,verification,/m/07j7r,1
...
```

In [8]:
# Define a schema for the data so the Confidence is a number, not a string
label_schema = StructType([
    StructField("ImageID", StringType()),
    StructField("Source", StringType()),
    StructField("LabelName", StringType()),
    StructField("Confidence", DoubleType())
])

# Read in the csv files using the schema
image_labels_1 = spark.read\
                    .csv('/data/google_open_image/labels/test-annotations-human-imagelabels-boxable.csv', 
                        header = True,
                        schema = label_schema)
image_labels_2 = spark.read\
                    .csv('/data/google_open_image/labels/train-annotations-human-imagelabels-boxable.csv', 
                        header = True,
                        schema = label_schema)
image_labels_3 = spark.read\
                    .csv('/data/google_open_image/labels/validation-annotations-human-imagelabels-boxable.csv', 
                        header = True,
                        schema = label_schema)

# join the 3 files into one large dataframe
image_labels = image_labels_1.union(image_labels_2).union(image_labels_3)

# Check the image labels dataframe

In [9]:
# Count the distinct number of image Ids. Should be 1.9 million
# Every image can have more than one label, so there are many more
# labels than image IDs.
image_labels.groupBy('Source')\
    .agg(F.countDistinct("ImageId").alias("n_images"))\
    .toPandas()

,Source,n_images
0,verification,1906725
1,crowdsource-verification,276745


# Filter image labels to labels of interest

Joining the `image_labels` with the `labels` dataframe adds the `LabelName` column to the larger `image_labels` dataframe.

Then filter to only high-confidence labels and just grab the image IDs of interest.

In [10]:
# join and filter
image_labels = image_labels.join(labels, on = 'LabelName', how = 'right')\
                .filter("Confidence > 0.99")

In [11]:
# Verify the dataframe. Count how many images we have after filtering.
image_labels.groupBy('Source')\
    .agg(F.countDistinct("ImageId").alias("n_images"))\
    .toPandas()

,Source,n_images
0,verification,3755


# Get just the image IDs of interest.

A dataframe of the distinct image IDs will be used to only get the needed images from the parquet file of all the images.

In [12]:
# distinct image IDs.
image_ids = image_labels.filter("Confidence > 0.99").select('ImageID').distinct()

In [13]:
# Check how many images there are.
image_ids.count()

3755

# Read the Parquet file containing the dataframe with the image data.

In [14]:
images_parquet = spark.read.parquet('/etl/google_open_image/images_coalesced.parquet')

In [15]:
# there's a lot of columns that aren't needed, select just the ones of interest.
images_parquet = images_parquet.select(['ImageID', 'Subset', 'Data'])\
                .withColumn("ImageID", F.lower(F.col('ImageID')))

In [16]:
# Verify the column names in the dataframe
images_parquet

DataFrame[ImageID: string, Subset: string, Data: binary]

# Filter images to just those of interest.

This is where we use the `image_ids` dataframe to filter the parquet file dataframe to just the data needed.

In [17]:
images_parquet = image_ids.join(images_parquet, on = 'ImageID', how = 'left')

In [18]:
# Verify the dataframe
images_parquet.count()

3755

# Bounding Boxes

Read in all the bounding box data similarly to how the image labels were read in. Notice that a schema wasn't used, which actually comes to bite us later.

In [19]:
# Read the 3 bounding box csv files.
bounding_boxes_1 = spark.read.csv('/data/google_open_image/bboxes/test-annotations-bbox.csv', header = True)
bounding_boxes_2 = spark.read.csv('/data/google_open_image/bboxes/train-annotations-bbox.csv', header = True)
bounding_boxes_3 = spark.read.csv('/data/google_open_image/bboxes/validation-annotations-bbox.csv', header = True)

# Join the dataframes into a single dataframe.
bounding_boxes = bounding_boxes_1.union(bounding_boxes_2).union(bounding_boxes_3)

In [20]:
# Verify the schema
bounding_boxes

DataFrame[ImageID: string, Source: string, LabelName: string, Confidence: string, XMin: string, XMax: string, YMin: string, YMax: string, IsOccluded: string, IsTruncated: string, IsGroupOf: string, IsDepiction: string, IsInside: string]

# Again, use the images_ids dataframe to filter to just the bounding boxes required.

In [21]:
# Join on ImageID to get just the bounding boxes we have image data for.
bbs = image_ids.join(bounding_boxes, on = 'ImageID', how = 'left')

In [22]:
# Join in the labels so there are human-readable labels on the bounding boxes.
bbs = labels.join(bbs, on = 'LabelName', how = 'left')

In [23]:
# Check how many boxes there are.
bbs.count()

10176

# Finally, we're ready to extract image chips from the images

This join actually makes copies of the image data for each bounding box. While this isn't ideal, it makes extracting the image chips defined by the bounding boxes easier to calculate.

In [24]:
image_chips = images_parquet.join(bbs, on = 'ImageID', how = 'right')

In [25]:
# Check how many chips we have to make sure the join was the correct one.
image_chips.count()

10176

In [26]:
# Verify the Schema
image_chips

DataFrame[ImageID: string, Subset: string, Data: binary, LabelName: string, LabelText: string, Source: string, Confidence: string, XMin: string, XMax: string, YMin: string, YMax: string, IsOccluded: string, IsTruncated: string, IsGroupOf: string, IsDepiction: string, IsInside: string]

# Cache the dataframe

Now that we have a dataframe of an appropriate size, cache it so future operations are faster.

In [27]:
#image_chips.cache()

Remember that the `cache()` operation isn't evaluated right away. The next operation on the dataframe will take longer because it must calculate the dataframe and store it in memory. After that, things shoud speed up. Let's test that theory out with a calculation.

Run the same calculation twice and compare the run times.

In [28]:
#import time
#
#begin = time.perf_counter()
#image_chips.groupby('LabelText').agg(F.countDistinct('ImageID').alias('n_images')).toPandas()
#end = time.perf_counter()
#
#print(f"First evaluation took {end - begin} seconds.")

In [29]:
#import time
#
#begin = time.perf_counter()
#image_chips.groupby('LabelText').agg(F.countDistinct('ImageID').alias('n_images')).toPandas()
#end = time.perf_counter()
#
#print(f"Second evaluation took {end - begin} seconds.")

As you can see the second run was significantly faster.

# Define a UDF to extract the portion of the image defined by the bounding box.

The bounding box coordinates are given as fractions of the number of pixels in the image. For example: XMin = 0.25 if the image has a width (X) of 1000 pixels, XMin is pixel 250.

In [30]:
def extract_chip(data, xmin, xmax, ymin, ymax):
    from PIL import Image
    import io, math
    
    # Read the image data using Pillow
    img = Image.open(io.BytesIO(data))
    # Get the size of the image 
    (width, height) = img.size
    
    # Calculate the bounding box pixels
    # observe the use of float function here. That's necessary
    # because the bounding box data were read in as strings, not doubles.
    left = math.floor(float(xmin)*width)
    upper = math.floor(float(ymin)*height)
    right = math.floor(float(xmax)*width)
    lower = math.floor(float(ymax)*height)
    
    # Crop the image to the bounding box size
    img = img.crop(box = (left, upper, right, lower))
    
    # Save the image to a byte-buffer
    buff = io.BytesIO()
    img.save(buff, format = "JPEG")
    
    # Get the raw bytes of the jpeg data.
    byte_array = buff.getvalue()
    return byte_array   # return buff.getvalue() doesn't work. This a quirk of pyspark not being able to determine the output type of a function call.

# Wrap the function as a spark udf (user-defined function) with a binary return type
udf_extract_chip = F.udf(extract_chip, returnType = BinaryType())

# Create a new column with the image chip data
image_chips = image_chips.withColumn("chip_data", udf_extract_chip("Data","XMin","XMax","YMin","YMax"))

In [31]:
# cache the chips
#image_chips.cache()

# Check the chips dataframe

In [32]:
# Force the cache to happen
image_chips.count()

10176

# Write Just the Chips to a Parquet File

In [33]:
image_chips = image_chips.drop('data')
image_chips.write.parquet("/user/klucar/image_chips_no_data.parquet")

AnalysisException: 'path hdfs://worker2.hdp-internal:8020/user/klucar/image_chips_no_data.parquet already exists.;'

# OFF THE RAILS

Some things got busy on the cluster! Here I wrote out the chip data to HDFS as files just to show how.

# To use the chips later, write them to HDFS

Here I use a different hdfs library. This is because I was unable to correctly configure `pyarrow` to work on the cluster nodes. See the new HDFS.ipynb file for details on this.

Instead of pulling every image chip back to the notebook and then uploading it to HDFS, create a UDF that writes a chip file to HDFS and evaluate it on the dataframe. This will write the chips from every spark node _in parallel_ across the cluster.

In [ ]:
# write chip to hdfs
def write_chip_hdfs(data, id, label):
    import io
    from random import randint
    
    from hdfs import InsecureClient
    client = InsecureClient('http://10.3.0.2:9870', user='klucar')
    
    filename = f"{label}_{id}_{randint(0,1000000)}.jpeg"
    path = "/user/klucar/keras_chips/" + filename
    client.write(path, io.BytesIO(data))
    
    return path

# wrap function in a udf
udf_write_chip_hdfs = F.udf(write_chip_hdfs, StringType())

# Call the UDF

```python
image_chips = image_chips.withColumn("hdfs_path", udf_write_chip_hdfs("chip_data", "ImageID", "LabelText"))
image_chips.agg(F.countDistinct('hdfs_path')).show()
```

In [ ]:
# evaluate the chips and examine some outputs.
#image_chips.limit(10).toPandas()

In [ ]:
#spark.stop()